In [617]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt
import math
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [618]:
ais_dataset = pd.read_csv('./datasets/ais_dataset.csv')
port_call_dataset = pd.read_csv('./datasets/port_call_dataset.csv')
distance_dataset = pd.read_csv('./datasets/distance_dataset.csv')
llaf_table = pd.read_csv('./datasets/llaf_table.csv')




print(len(ais_dataset['imo'].unique()))
print(len(port_call_dataset['imo'].unique()))
print(ais_dataset.info())
print(port_call_dataset.info())
print(distance_dataset.info())
# no terminal, maneuvering , berth
# suggests that only anchored and transit
# we can simply find if anchored or not

2543
4166
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251769 entries, 0 to 251768
Data columns (total 30 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   imo                    251769 non-null  int64  
 1   mmsi                   251769 non-null  int64  
 2   vessel_name            251769 non-null  object 
 3   date_of_build          251769 non-null  object 
 4   vessel_type            251769 non-null  object 
 5   group                  251769 non-null  object 
 6   timestamp              251769 non-null  object 
 7   lon                    251769 non-null  float64
 8   lat                    251769 non-null  float64
 9   nav_stat               251769 non-null  int64  
 10  speed                  249752 non-null  float64
 11  course                 251769 non-null  int64  
 12  heading                250464 non-null  float64
 13  fuel_category          251769 non-null  int64  
 14  main_engine_fuel_type  251

### Dropping Columns
---

##### AIS

In [619]:
# dropping missing values
ais_dataset.drop(columns=['berth' , 'terminal' , 'maneuvering_zone'] , inplace=True)
# ais_dataset.dropna(inplace=True) # 《 5% missing data

In [620]:
# From background research cum provided information
# ABL AEL !== 0
# drop meta data:
# port_name , mmsi (cannot be used since other datasets only have imo) , date_of_build
useless_columns = ['port_name' , 'mmsi' , 'date_of_build']
ais_dataset.drop(columns=useless_columns , inplace=True)

In [621]:
abl_ael_not_zero_condition = (ais_dataset['abl'] > 0) & (ais_dataset['ael'] > 0)
ais_dataset = ais_dataset[abl_ael_not_zero_condition]

In [622]:
ais_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 248013 entries, 0 to 251768
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   imo                    248013 non-null  int64  
 1   vessel_name            248013 non-null  object 
 2   vessel_type            248013 non-null  object 
 3   group                  248013 non-null  object 
 4   timestamp              248013 non-null  object 
 5   lon                    248013 non-null  float64
 6   lat                    248013 non-null  float64
 7   nav_stat               248013 non-null  int64  
 8   speed                  246213 non-null  float64
 9   course                 248013 non-null  int64  
 10  heading                246742 non-null  float64
 11  fuel_category          248013 non-null  int64  
 12  main_engine_fuel_type  248013 non-null  object 
 13  aux_engine_fuel_type   248013 non-null  object 
 14  engine_type            247590 non-null  o

### Imputing

In [623]:
# When looking through the speed we notice each imo has a null speed
# This suggests that each ship is anchored-travels to sg-travels further/stays in sg
# Speed null values can be imputed as '0'
ais_dataset['speed'].fillna(0 , inplace=True)

In [624]:
# We can also drop speeds that are greather than 30
less_than_30_condition = ais_dataset['speed'] < 30
ais_dataset = ais_dataset[less_than_30_condition]

In [625]:
ais_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 248013 entries, 0 to 251768
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   imo                    248013 non-null  int64  
 1   vessel_name            248013 non-null  object 
 2   vessel_type            248013 non-null  object 
 3   group                  248013 non-null  object 
 4   timestamp              248013 non-null  object 
 5   lon                    248013 non-null  float64
 6   lat                    248013 non-null  float64
 7   nav_stat               248013 non-null  int64  
 8   speed                  248013 non-null  float64
 9   course                 248013 non-null  int64  
 10  heading                246742 non-null  float64
 11  fuel_category          248013 non-null  int64  
 12  main_engine_fuel_type  248013 non-null  object 
 13  aux_engine_fuel_type   248013 non-null  object 
 14  engine_type            247590 non-null  o

In [626]:
# imputing the max speed for the distance dataset
# Adding vref to the dataset
# we will calculate for null max speeds to be vref * 1.06

distance_dataset = pd.merge(distance_dataset , ais_dataset[['imo' , 'vref']] , on='imo' , how='inner')
distance_dataset = distance_dataset.drop_duplicates()

distance_dataset['max_speed_kt'] = distance_dataset.apply(lambda row: row['vref'] * 1.06 if pd.isna(row['max_speed_kt']) else row['max_speed_kt'] , axis=1)

### Feature Engineering

port call

In [627]:
# creating time col (duration spent during an event)
# duration travelled = current - prev earliest time
# duration in port = current latest - current earliest
def get_time_difference_in_seconds_from_utc(s1 , s2):
  try: 
    current_time_timestamp = dt.strptime(s1, '%Y-%m-%dT%H:%M:%S.%f%z').timestamp()
    prev_time_timestamp = dt.strptime(s2, '%Y-%m-%dT%H:%M:%S.%f%z').timestamp()
    duration_timestamp_seconds = abs(current_time_timestamp - prev_time_timestamp)
    return duration_timestamp_seconds
  except:
    # For when value is null
    return np.nan

def get_duration(col1 , col2):
  def calculate(rows):
    s1 = rows[col1]
    s2 = rows[col2]
    return round(get_time_difference_in_seconds_from_utc(s1 , s2) / 3600 , 5) # rounding to 5 dp
  
  return calculate


prev_anchor = ['prev_country_earliest_visit_time_utc' , 'prev_country_latest_visit_time_utc']
prev_to_sg_transit = ['prev_country_latest_visit_time_utc' , 'current_earliest_visit_time_utc']
sg_anchor = ['current_latest_visit_time_utc' , 'current_earliest_visit_time_utc']
sg_to_next_transit = ['current_latest_visit_time_utc' , 'next_country_earliest_visit_time_utc']
next_anchor = ['next_country_earliest_visit_time_utc' , 'next_country_latest_visit_time_utc']

port_call_dataset['Prev Anchor'] = port_call_dataset.loc[: , prev_anchor].apply(get_duration(*prev_anchor) , axis=1)
port_call_dataset['Prev to SG Transit'] = port_call_dataset.loc[: , prev_to_sg_transit].apply(get_duration(*prev_to_sg_transit), axis=1) 
port_call_dataset['SG Anchor'] = port_call_dataset.loc[: , sg_anchor].apply(get_duration(*sg_anchor), axis=1)
port_call_dataset['SG to Next Transit'] = port_call_dataset.loc[: , sg_to_next_transit].apply(get_duration(*sg_to_next_transit) , axis=1)
port_call_dataset['Next Anchor'] = port_call_dataset.loc[: , next_anchor].apply(get_duration(*next_anchor) , axis=1)

AIS dataset

In [628]:
def get_state(row):
  TRANSIT_NAV_STAT = [0 , 3 , 4 , 8 , 11 , 12]
  if row['speed'] > 1 or row['nav_stat'] in TRANSIT_NAV_STAT:
    return 'Transit'
  return 'Anchored'

In [629]:
ais_dataset['state'] = ais_dataset.apply(get_state , axis=1)

In [630]:
ais_dataset['state']

0         Transit
1         Transit
2         Transit
3         Transit
4         Transit
           ...   
251764    Transit
251765    Transit
251766    Transit
251767    Transit
251768    Transit
Name: state, Length: 248013, dtype: object

In [631]:
# Adding the admission tier


### Calculating Before Emissions

In [632]:
# Calculating ef
ais_dataset['ef_me'] = ais_dataset['sfc_me'] * 0.867 * 3.667
ais_dataset['ef_ae'] = ais_dataset['sfc_ae'] * 0.867 * 3.667
ais_dataset['ef_ab'] = ais_dataset['sfc_ab'] * 0.867 * 3.667


In [633]:
# result["ratio"] = result["max_speed_kt"] / result["vref"]
# result["rounded"] = result["ratio"].apply(lambda x: 0 if x < 1 else 1)

# # Count the number of 0s and 1s
# counts = result["rounded"].value_counts()

# # Plot the pie chart
# plt.figure(figsize=(8, 6))
# counts.plot.pie(
#     labels=["Below 1 (0)", "Above or Equal to 1 (1)"],
#     autopct="%1.1f%%",
#     startangle=90,
#     colors=["lightblue", "orange"]
# )

# # Add a title
# plt.title("Distribution of Rounded Values", fontsize=14)
# plt.ylabel("")  # Hide the y-axis label
# plt.show()

In [653]:
print(np.mean(port_call_dataset['SG Anchor']))
print(np.mean(port_call_dataset['Prev Anchor']))
print(np.mean(port_call_dataset['Next Anchor'].notnull()))
print(np.sum([3.9299050600096015,
1.5400323979836774,
0.971435429668747]))

3.9299050600096015
1.5400323979836774
0.971435429668747
6.441372887662026


In [635]:
print(np.sort(port_call_dataset['current_earliest_visit_time_utc'].unique()))
print(ais_dataset.columns)

['2024-06-30T00:20:08.000+0000' '2024-06-30T00:20:42.000+0000'
 '2024-06-30T00:45:03.000+0000' ... '2024-07-31T22:41:13.000+0000'
 '2024-07-31T23:15:01.000+0000' '2024-07-31T23:45:02.000+0000']
Index(['imo', 'vessel_name', 'vessel_type', 'group', 'timestamp', 'lon', 'lat',
       'nav_stat', 'speed', 'course', 'heading', 'fuel_category',
       'main_engine_fuel_type', 'aux_engine_fuel_type', 'engine_type',
       'anchorage', 'p', 'vref', 'sfc_me', 'sfc_ae', 'sfc_ab', 'ael', 'abl',
       'distance', 'state', 'ef_me', 'ef_ae', 'ef_ab'],
      dtype='object')


In [636]:
avgtime = pd.merge(port_call_dataset, ais_dataset[['imo', 'vessel_type']], how='inner', on='imo')
# Check the columns of the merged DataFrame to ensure 'vessel_type' is present
print(avgtime.columns)

# Now group by 'vessel_type' and calculate the mean of 'Sg Anchor'
avg_time_per_vessel_type = avgtime.groupby('vessel_type')['SG Anchor'].mean()
avg_speed_per_vessel_type = avgtime.groupby('vessel_type')['Prev to SG Transit'].mean()# Print the result
print(avg_time_per_vessel_type.nlargest(10))
print(avg_speed_per_vessel_type.nlargest(10))

Index(['current_snapshot_date_utc', 'imo', 'current_port_name',
       'current_port_country_name', 'current_earliest_visit_time_utc',
       'current_latest_visit_time_utc', 'current_zone_polygon_name',
       'current_zone_type', 'next_port_name', 'next_country_name',
       'next_country_earliest_visit_time_utc',
       'next_country_latest_visit_time_utc', 'next_zone_polygon_name',
       'next_snapshot_date', 'next_zone_type', 'prev_port_name',
       'prev_country_name', 'prev_country_earliest_visit_time_utc',
       'prev_country_latest_visit_time_utc', 'prev_zone_polygon_name',
       'prev_snapshot_date', 'prev_zone_type', 'Prev Anchor',
       'Prev to SG Transit', 'SG Anchor', 'SG to Next Transit', 'Next Anchor',
       'vessel_type'],
      dtype='object')
vessel_type
Deck Cargo Ship              22.287801
Shuttle Tanker               17.537780
Livestock Carrier            17.138988
Heavy Load Carrier           16.533120
LNG Tanker                   16.162102
Crude Oil Tank

### Attaching Emissions to port call

In [637]:
def calculate_emissions_aux(row):
    LA = row['ael']
    LBL = row['abl']
    EF_SFCAE = row['ef_ae']
    EF_SFCAB = row['ef_ab']
    base_emissions = (LA * EF_SFCAE + LBL * EF_SFCAB)
    return base_emissions


# Apply emissions calculations for 'Aux' and 'Transit' columns
ais_dataset['Aux'] = ais_dataset.apply(calculate_emissions_aux, axis=1)


In [638]:
ais_dataset['Aux_total'] = ais_dataset.groupby('imo')['Aux'].transform('sum')


In [639]:
print(ais_dataset['Aux_total'])


0         6.868089e+07
1         6.868089e+07
2         6.868089e+07
3         6.868089e+07
4         6.868089e+07
              ...     
251764    1.313553e+08
251765    1.313553e+08
251766    1.313553e+08
251767    1.313553e+08
251768    1.313553e+08
Name: Aux_total, Length: 248013, dtype: float64


In [640]:
def calculate_emissions_transit(row):
    try:
        if row['state'] != 'Transit':
            return 0

        IMO = row['imo']
        transit_and_imo_condition = (ais_dataset['state'] == 'Transit') & (ais_dataset['imo'] == IMO)

        speeds_in_transit = ais_dataset[transit_and_imo_condition]['speed']
        if speeds_in_transit.empty:
            return 0
        
        AVERAGE_SPEED = speeds_in_transit.mean()

        max_speed_info = distance_dataset[distance_dataset['imo'] == IMO]
        if max_speed_info.empty:
            return 0
        MAX_SPEED = max_speed_info['max_speed_kt'].values[0]

        LF = (AVERAGE_SPEED / MAX_SPEED) ** 3

        EF_SFME = row['ef_me']
        P = row['p']

        load_factor = f'{math.ceil(LF * 100)}%'
        LLAF = llaf_table[llaf_table['Load'] == load_factor]

        if LLAF.empty:
            LLAF = 1
        else:
            LLAF = LLAF.iloc[0]['CO2']

        emissions = P * LF * EF_SFME * LLAF
        return emissions
    except Exception as e:
        print(f"Error processing IMO {row['imo']}: {e}")
        return 0

# Apply the function to the dataset and test it with print statements
ais_dataset['Transit'] = ais_dataset.apply(calculate_emissions_transit, axis=1)

In [655]:

a= llaf_table[llaf_table['Load'] == '2%']['CO2']


In [656]:
df = ais_dataset.copy()
df

,imo,vessel_name,vessel_type,group,timestamp,lon,lat,nav_stat,speed,course,...,ael,abl,distance,state,ef_me,ef_ae,ef_ab,Aux,Aux_total,Transit
0,9984730,SEACON ANTWERP,Bulk Carrier,Bulk Carrier,2024-07-28T00:23:32.000Z,104.029630,1.283717,0,0.0,3,...,316.0,138.0,6.221528,Transit,531.259192,664.789330,953.7867,341695.992848,6.868089e+07,194.234766
1,9984730,SEACON ANTWERP,Bulk Carrier,Bulk Carrier,2024-07-28T00:25:02.000Z,104.029755,1.285643,0,4.0,3,...,316.0,138.0,0.115910,Transit,531.259192,664.789330,953.7867,341695.992848,6.868089e+07,194.234766
2,9984730,SEACON ANTWERP,Bulk Carrier,Bulk Carrier,2024-07-28T00:30:01.000Z,104.029420,1.291690,0,4.0,342,...,316.0,138.0,0.363603,Transit,531.259192,664.789330,953.7867,341695.992848,6.868089e+07,194.234766
3,9984730,SEACON ANTWERP,Bulk Carrier,Bulk Carrier,2024-07-28T00:32:51.000Z,104.028160,1.294758,0,4.0,328,...,316.0,138.0,0.199119,Transit,531.259192,664.789330,953.7867,341695.992848,6.868089e+07,194.234766
4,9984730,SEACON ANTWERP,Bulk Carrier,Bulk Carrier,2024-07-28T00:35:02.000Z,104.026474,1.296387,0,3.0,304,...,316.0,138.0,0.140720,Transit,531.259192,664.789330,953.7867,341695.992848,6.868089e+07,194.234766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251764,9739953,PACIFIC QINGDAO,LPG Tanker,LPG Tanker,2024-07-09T21:41:15.000Z,103.684640,1.188230,0,2.0,256,...,536.0,367.0,0.053041,Transit,525.504679,650.482529,953.7867,698698.354658,1.313553e+08,28811.330075
251765,9739953,PACIFIC QINGDAO,LPG Tanker,LPG Tanker,2024-07-09T21:45:04.000Z,103.681130,1.187597,0,3.0,264,...,536.0,367.0,0.214073,Transit,525.504679,650.482529,953.7867,698698.354658,1.313553e+08,28811.330075
251766,9739953,PACIFIC QINGDAO,LPG Tanker,LPG Tanker,2024-07-09T21:45:37.000Z,103.681800,1.187628,0,3.0,267,...,536.0,367.0,0.040347,Transit,525.504679,650.482529,953.7867,698698.354658,1.313553e+08,28811.330075
251767,9739953,PACIFIC QINGDAO,LPG Tanker,LPG Tanker,2024-07-09T21:50:03.000Z,103.673874,1.188243,0,6.0,280,...,536.0,367.0,0.477266,Transit,525.504679,650.482529,953.7867,698698.354658,1.313553e+08,28811.330075


In [657]:
grouped_df_aux = df.groupby(by='imo')['Aux'].sum()
port_call_dataset['Total Anchor'] = port_call_dataset.apply(lambda row: row['SG Anchor'] + row['Prev Anchor'] + row['Next Anchor'] , axis=1)
port_call_dataset['Total Anchor']

0        1.03305
1       40.91723
2        0.17945
3       40.25527
4       11.19112
          ...   
4161    21.01139
4162    15.31195
4163     0.50778
4164    19.06416
4165    33.62556
Name: Total Anchor, Length: 4166, dtype: float64

In [658]:
merged_df_aux = pd.merge(grouped_df_aux , port_call_dataset[['imo' , 'Total Anchor']] , how='inner' , on='imo')
merged_df_aux = merged_df_aux['Aux'] * merged_df_aux['Total Anchor']

In [659]:
merged_df_aux.sum()

3571233126960.708

In [660]:
grouped_df_transit = df[df['state'] == 'Transit']
grouped_df_transit = grouped_df_transit.groupby(by='imo')[['Aux' , 'Transit']].sum()


In [661]:

port_call_dataset['Total Transit'] = port_call_dataset.apply(lambda row: row['Prev to SG Transit'] + row['SG to Next Transit'] , axis=1)
port_call_dataset['Total Transit']

0         0.37611
1         1.87639
2        94.41195
3       192.13361
4        31.28917
          ...    
4161      3.93917
4162      4.16834
4163     80.27611
4164      9.90111
4165      2.69194
Name: Total Transit, Length: 4166, dtype: float64

In [ ]:

merged_df_transit = pd.merge(grouped_df_transit , port_call_dataset[['imo' , 'Total Transit']] , on='imo' , how='inner')
merged_df_transit


,imo,Aux,Transit,Total Transit
0,1013327,1.357429e+06,2.661224e+05,0.37611
1,1014838,7.053960e+06,2.239151e+05,3.57945
2,1015820,4.756757e+06,4.473113e+03,1.87639
3,1017775,9.051436e+05,0.000000e+00,1.84056
4,1018547,2.160499e+07,1.963012e+06,298.82056
...,...,...,...,...
2191,9991082,6.823390e+05,5.656263e+05,2.81945
2192,9991109,1.266088e+07,0.000000e+00,3.93917
2193,9991252,1.384008e+06,6.004896e+04,4.16834
2194,9994917,2.031756e+06,3.813852e+05,80.27611


In [663]:
total_transit = (merged_df_transit['Aux'] + merged_df_transit['Transit']) * merged_df_transit['Total Transit']
total_transit.sum()

5891226742146.073

### Propose a 5% reduction in timing

In [664]:
port_call_dataset['Total Anchor'] = port_call_dataset['Total Anchor'] * 0.95
port_call_dataset['Total Transit'] = port_call_dataset['Total Transit'] * 0.95

In [665]:
grouped_df_aux = df.groupby(by='imo')['Aux'].sum()

In [666]:
merged_df_aux = pd.merge(grouped_df_aux , port_call_dataset[['imo' , 'Total Anchor']] , how='inner' , on='imo')
merged_df_aux = merged_df_aux['Aux'] * merged_df_aux['Total Anchor']

In [667]:
merged_df_aux.sum()

3392671470612.673

In [668]:
grouped_df_transit = df[df['state'] == 'Transit']
grouped_df_transit = grouped_df_transit.groupby(by='imo')[['Aux' , 'Transit']].sum()

In [669]:

merged_df_transit = pd.merge(grouped_df_transit , port_call_dataset[['imo' , 'Total Transit']] , on='imo' , how='inner')
merged_df_transit


,imo,Aux,Transit,Total Transit
0,1013327,1.357429e+06,2.661224e+05,0.357304
1,1014838,7.053960e+06,2.239151e+05,3.400478
2,1015820,4.756757e+06,4.473113e+03,1.782570
3,1017775,9.051436e+05,0.000000e+00,1.748532
4,1018547,2.160499e+07,1.963012e+06,283.879532
...,...,...,...,...
2191,9991082,6.823390e+05,5.656263e+05,2.678477
2192,9991109,1.266088e+07,0.000000e+00,3.742211
2193,9991252,1.384008e+06,6.004896e+04,3.959923
2194,9994917,2.031756e+06,3.813852e+05,76.262304


In [670]:

total_transit = (merged_df_transit['Aux'] + merged_df_transit['Transit']) * merged_df_transit['Total Transit']
total_transit.sum()

5596665405038.77